In [5]:
!pip install simpletransformers

In [1]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd

train_df = pd.read_csv("tweet_emotions.csv")

In [2]:
train_df['sentiment'].unique()

array(['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger'],
      dtype=object)

In [3]:
train_df['text']=train_df['content']
train_df['sentiments']=train_df['sentiment']

In [4]:
del train_df['tweet_id']
del train_df['sentiment']
del train_df['content']

In [5]:
train_df.size

80000

In [6]:
train_df = train_df.rename(columns={'sentiments':'labels'})

encoding_labels = {'empty': 0, 'sadness':1, 'enthusiasm':2, 'neutral':3, 'worry':4, 'surprise':5,
       'love': 6, 'fun': 7, 'hate': 8, 'happiness': 9, 'boredom': 10, 'relief' : 11, 'anger': 12}

train_df['labels'] = train_df['labels'].map(encoding_labels)
train_df



,text,labels
0,@tiffanylue i know i was listenin to bad habi...,0
1,Layin n bed with a headache ughhhh...waitin o...,1
2,Funeral ceremony...gloomy friday...,1
3,wants to hang out with friends SOON!,2
4,@dannycastillo We want to trade with someone w...,3
...,...,...
39995,@JohnLloydTaylor,3
39996,Happy Mothers Day All my love,6
39997,Happy Mother's Day to all the mommies out ther...,6
39998,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...,9


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /home/yash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
#  Data Preprocessing 

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
train_df['text'] = train_df['text'].apply(clean_text)
train_df


,text,labels
0,tiffanylue know listenin bad habit earlier sta...,0
1,layin n bed headache ughhhhwaitin call,1
2,funeral ceremonygloomy friday,1
3,wants hang friends soon,2
4,dannycastillo want trade someone houston ticke...,3
...,...,...
39995,johnlloydtaylor,3
39996,happy mothers day love,6
39997,happy mothers day mommies woman man long youre...,6
39998,niariley wassup beautiful follow peep new hit ...,9


In [23]:
# Spliitting the dataset into training and test

X = train_df.text
y = train_df.labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)
X.size

40000

In [ ]:
#  Bert Model

# model_args = ClassificationArgs(num_train_epochs=3, overwrite_output_dir= True)

# model = ClassificationModel('bert','bert-base-cased',num_labels=13,args=model_args) 
# model.train_model(train_df)

In [10]:
# Logistic Regression

%%time
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5,max_iter=100)),
               ])
logreg.fit(X_train, y_train)






CPU times: user 1min 25s, sys: 1min 32s, total: 2min 58s
Wall time: 24.9 s


/home/yash/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=100000.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=

In [12]:
from sklearn.metrics import classification_report
y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))


accuracy 0.2628333333333333


In [24]:
# Linear Support Vector Machine ( SGD Classifier)

from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X, y)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                      

In [25]:
y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

accuracy 0.8030833333333334


In [27]:
import pickle
import requests
import json
pickle.dump(sgd, open('model2.pkl','wb'))